In [1]:
try:
    import lovely_tensors as lt
except:
    ! pip install --upgrade lovely-tensors
    import lovely_tensors as lt
    
lt.monkey_patch()

from pathlib import Path
Path.ls = lambda x: list(x.iterdir())

In [2]:
ldm_cfg_path = Path('models/ldm/cin256/config.yaml')
ldm_cfg_path.exists()

True

In [3]:
import os
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from hydra.utils import instantiate
from omegaconf import OmegaConf

with initialize(version_base=None, config_path=str(ldm_cfg_path.parent)):
    cfg = compose(config_name='config')

In [4]:
from ldm.util import instantiate_from_config

In [25]:
device = 'cuda:0'

In [26]:
model = instantiate_from_config(cfg.model)
model.eval()
model = model.to(device)
1

LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 394.98 M params.
Keeping EMAs of 628.
making attention of type 'vanilla' with 512 in_channels
making attention of type 'vanilla' with 512 in_channels
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels
making attention of type 'vanilla' with 512 in_channels
making attention of type 'vanilla' with 512 in_channels
making attention of type 'vanilla' with 512 in_channels
Restored from /app/notebooks/03_ldm_vae/latent-diffusion/logs/2023-05-18T12-05-30_config/checkpoints/last.ckpt with 0 missing and 50 unexpected keys


1

In [27]:
import torch

x = {
    "image": torch.rand(4, 256, 256, 48).to(device),
    "class_label": torch.tensor([42, 42, 42, 42]).to(device),
    "human_label": ['054656345', '054656345', '054656345', '054656345']
}
x

{'image': tensor[4, 256, 256, 48] n=12582912 (48Mb) x∈[5.960e-08, 1.000] μ=0.500 σ=0.289 cuda:0,
 'class_label': tensor[4] i64 x∈[42, 42] μ=42.000 σ=0. cuda:0 [42, 42, 42, 42],
 'human_label': ['054656345', '054656345', '054656345', '054656345']}

In [49]:
log_res = model.log_images(x, ddim_steps=2)

Plotting: Switched to EMA weights
Data shape for DDIM sampling is (4, 4, 32, 32), eta 1.0
Running DDIM Sampling with 2 timesteps


DDIM Sampler: 100%|██████████| 2/2 [00:00<00:00, 18.01it/s]

Plotting: Restored training weights


Plotting Quantized Denoised: Switched to EMA weights
Data shape for DDIM sampling is (4, 4, 32, 32), eta 1.0
Running DDIM Sampling with 2 timesteps


DDIM Sampler: 100%|██████████| 2/2 [00:00<00:00, 17.72it/s]

...

Plotting Outpaint: Restored training weights


Plotting Progressives: Switched to EMA weights


Progressive Generation: 100%|██████████| 1000/1000 [00:54<00:00, 18.48it/s]


Plotting Progressives: Restored training weights


Progressive Generation: 100%|██████████| 6/6 [00:00<00:00, 17.98it/s]


In [53]:
def b_triplane2rgb(x, mult=20):
    feat_size = x.shape[1] // 3
    xy_plane = x[:, :feat_size].mean(dim=1, keepdim=True).repeat(1, 3, 1, 1)
    xz_plane = x[:, feat_size:feat_size*2].mean(dim=1, keepdim=True).repeat(1, 3, 1, 1)
    yz_plane = x[:, feat_size*2:].mean(dim=1, keepdim=True).repeat(1, 3, 1, 1)

    triplane = torch.cat([xy_plane, xz_plane, yz_plane], dim=-1)*mult
    return triplane

def triplane2rgb(x, mult=20):
    feat_size = x.shape[0] // 3
    xy_plane = x[:feat_size].mean(dim=0, keepdim=True).repeat(1, 3, 1, 1)
    # xz_plane = x[feat_size:feat_size*2].mean(dim=1, keepdim=True).repeat(1, 3, 1, 1)
    # yz_plane = x[feat_size*2:].mean(dim=1, keepdim=True).repeat(1, 3, 1, 1)

    triplane = torch.cat([xy_plane], dim=-1)*mult
    return triplane

In [55]:
for key, val in log_res.items():
    if val.shape[1] == 48:
        log_res[key] = b_triplane2rgb(val)
    if val.shape[0] == 48:
        log_res[key] = triplane2rgb(val)

In [56]:
log_res

{'inputs': tensor[4, 3, 256, 768] n=2359296 (9Mb) x∈[3.575, 16.759] μ=9.999 σ=1.443 cuda:0,
 'reconstruction': tensor[4, 3, 256, 768] n=2359296 (9Mb) x∈[0.112, 0.309] μ=0.195 σ=0.013 cuda:0,
 'conditioning': tensor[4, 3, 256, 256] f64 n=786432 (6Mb) x∈[-1.000, 1.000] μ=0.996 σ=0.080,
 'diffusion_row': tensor[1, 3, 1034, 1550] n=4808100 (18Mb) x∈[-0.050, 0.473] μ=0.188 σ=0.055 cuda:0,
 'samples': tensor[4, 3, 256, 768] n=2359296 (9Mb) x∈[-0.195, 0.477] μ=0.186 σ=0.065 cuda:0,
 'samples_x0_quantized': tensor[4, 3, 256, 768] n=2359296 (9Mb) x∈[-0.141, 0.473] μ=0.181 σ=0.063 cuda:0,
 'samples_inpainting': tensor[4, 3, 256, 768] n=2359296 (9Mb) x∈[-0.111, 0.398] μ=0.186 σ=0.034 cuda:0,
 'mask': tensor[4, 1, 32, 32] n=4096 (16Kb) x∈[0., 1.000] μ=0.750 σ=0.433 cuda:0,
 'samples_outpainting': tensor[4, 3, 256, 768] n=2359296 (9Mb) x∈[-0.176, 0.400] μ=0.186 σ=0.033 cuda:0,
 'progressive_row': tensor[1, 3, 1034, 1550] n=4808100 (18Mb) x∈[-0.052, 0.464] μ=0.195 σ=0.061 cuda:0}

In [33]:
log_res.items()

{'inputs': tensor[4, 48, 256, 256] n=12582912 (48Mb) x∈[5.960e-08, 1.000] μ=0.500 σ=0.289 cuda:0,
 'reconstruction': tensor[4, 48, 256, 256] n=12582912 (48Mb) x∈[-0.005, 0.029] μ=0.010 σ=0.004 cuda:0,
 'conditioning': tensor[4, 3, 256, 256] f64 n=786432 (6Mb) x∈[-1.000, 1.000] μ=0.996 σ=0.080,
 'diffusion_row': tensor[48, 1034, 1550] n=76929600 (0.3Gb) x∈[-0.063, 0.092] μ=0.009 σ=0.006 cuda:0,
 'samples': tensor[4, 48, 256, 256] n=12582912 (48Mb) x∈[-0.058, 0.076] μ=0.009 σ=0.006 cuda:0,
 'samples_x0_quantized': tensor[4, 48, 256, 256] n=12582912 (48Mb) x∈[-0.062, 0.084] μ=0.009 σ=0.006 cuda:0,
 'samples_inpainting': tensor[4, 48, 256, 256] n=12582912 (48Mb) x∈[-0.047, 0.056] μ=0.009 σ=0.005 cuda:0,
 'mask': tensor[4, 1, 32, 32] n=4096 (16Kb) x∈[0., 1.000] μ=0.750 σ=0.433 cuda:0,
 'samples_outpainting': tensor[4, 48, 256, 256] n=12582912 (48Mb) x∈[-0.046, 0.056] μ=0.009 σ=0.005 cuda:0,
 'progressive_row': tensor[48, 1034, 1550] n=76929600 (0.3Gb) x∈[-0.063, 0.092] μ=0.009 σ=0.006 cuda: